In [1]:
import os
import glob
from tika import parser
import tika
#tika.TikaClientOnly = True

In [ ]:
def pdfToText(research_paper_name):
    #Create directory for the text files
    txtdir = 'C:/Users/Evan/Documents/7180QueryTool/DataTxt/'
    if (not os.path.exists(txtdir)):
        os.mkdir(txtdir)
    
    research_paper_data = parser.from_file(research_paper_name)
    research_paper_content = research_paper_data['content']
    research_paper_txt_name = research_paper_name.replace("pdf","txt")
    research_paper_txt_name = research_paper_txt_name.replace("Data","Datatxt")
    print(research_paper_content)
    output_file = open(research_paper_txt_name, "w+")
    output_file.write(research_paper_content)
    
    
    #text = pdfminer.high_level.extract_text(research_paper_name, codec='utf-8')
    #print(text)
    #research_paper_txt_name = research_paper_name.replace("pdf","txt")
    #! python pdftotext.py research_paper_txt_name
    #print(research_paper_txt_name)
    #output_file = open(research_paper_txt_name,"w+")
    #output_file.write(text)

In [61]:
def snippetToCsv(researchFile, sentenceNum):
  try:
    #Calling Spacy function to return a list
    #sentenceList = spacyFunction(researchFile,sentenceNum)
    #Test List
    sentenceList = ["I am a human.We all are humans.You are a human","Hello world. Hi. Test World"]

    #Splitting file name and path
    filePath, fileName = os.path.split(researchFile)
    print(fileName)
    print(filePath)

    #Creating a separate folder for all the csv files if it does not exist
    snippetDataDirPath = filePath + "/snippet_data"
    print(snippetDataDirPath)

    # Creating a folder for all the csv files if it does not exist
    if( not os.path.exists(snippetDataDirPath)):
      os.mkdir(snippetDataDirPath)

    #create a file name : Eg BestModel_3.csv
    fileName = fileName.replace(".txt","")
    fileName = fileName +"_"+ str(sentenceNum) + ".csv"
    print(fileName)

    #Complete path
    fileName = snippetDataDirPath+"/"+fileName

    #Checking if the files already exist in snippet_data
    if(glob.glob(fileName)):
      print("File already exists.")
      return

    #If not, we create and write in the files
    with open(fileName,"w+") as file: 
      for sentence in sentenceList:
        file.write(sentence)
        file.write("\n")
        file.write("<EOS>")
        file.write("\n")
    
  except Exception as e:
    print ("Unexpected error occurred : Details are ", sys.exc_info()[0])
  finally:
    print("Exit")

#Calling the function, 
#Takes research file(alongwith the file path) and number of sentences 
#snippetToCsv("/content/a.txt",3)


a.txt
/content
/content/snippet_data
a_3.csv
Exit
